## PREPROCESSING DATA

In [1]:
from preprocessing import Parser, ReportPreprocessor, SrcPreprocessor
from datasets import DATASET

from collections import defaultdict

In [2]:
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\nhatm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\nhatm\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [ ]:
parser = Parser(DATASET)
bug_reports = parser.report_parser()
src_files = parser.src_parser()

In [4]:
src_processor = SrcPreprocessor(src_files, parser.name).preprocess()
report_processor = ReportPreprocessor(bug_reports, parser.name).preprocess()

## INDEXING SOURCE CODE FILES


In [3]:
import pandas as pd
import ast

In [4]:
file_path = "outputs/tomcat_source_code_data.csv"
src_df = pd.read_csv(file_path)

In [6]:
def build_inverted_index(documents):
    inverted_index = defaultdict(list)
    for key, contents in documents:
        for content in contents:
            inverted_index[content].append(key)
    return inverted_index

In [7]:
src_df['all_content'] = src_df['all_content'].apply(ast.literal_eval)
src_df['all_content'] = src_df['all_content'].apply(lambda x: x['stemmed'])

In [8]:
data_src_codes_content = src_df[['key', 'all_content']].values.tolist()
data_src_codes_size = src_df[['key', 'total_lines']].values.tolist()

In [9]:
inverted_index_source_codes = build_inverted_index(data_src_codes_content)

In [10]:
len(inverted_index_source_codes)

23097

## BUILD VSM

In [11]:
import math

In [12]:
def compute_ltc(documents, inverted_index):
    tf_idf = defaultdict(lambda: defaultdict(float))
    total_documents = len(documents)

    for key, contents in documents:
        content_frequency = defaultdict(int)
        for content in contents:
            content_frequency[content] += 1

        for content, count in content_frequency.items():
            tf = 1 + math.log(count, 10)
            df = len(inverted_index.get(content, []))
            idf = math.log((total_documents + 1) / (df + 1), 10)


            tf_idf[key][content] = tf * idf

    for doc_id in tf_idf:
        norm = math.sqrt(sum(weight ** 2 for weight in tf_idf[doc_id].values()))
        if norm > 0:
            for content in tf_idf[doc_id]:
                tf_idf[doc_id][content] /= norm

    return tf_idf

In [13]:
vsm = compute_ltc(data_src_codes_content, inverted_index_source_codes)

## CALCULATE COEFFICIENT FILE SIZE

In [14]:
max_size = 0
min_size = math.inf
for data_size in data_src_codes_size:
    if data_size[1] > max_size:
        max_size = data_size[1]
    if data_size[1] < min_size:
        min_size = data_size[1]

In [15]:
def calculate_coefficient_size(size):
    value_standardization = (size - min_size) / (max_size - min_size)
    return 1/(1 + math.exp(-value_standardization))

In [16]:
coefficient_size = defaultdict(float)
for data_size in data_src_codes_size:
    coefficient_size[data_size[0]] = calculate_coefficient_size(data_size[1])

## HANDLE REPORT DATA

In [23]:
file_path = "outputs/tomcat_bug_reports.csv"
report_df = pd.read_csv(file_path)

In [27]:
report_df.iloc[4]

key                                                                   56010
summary                   {'stemmed': ['bug', 'jspfactori', 'getpagecont...
description                                {'stemmed': [], 'unstemmed': []}
fixed_files               java\org\apache\jasper\runtime\PageContextImpl...
report_time                                             2014-01-15 03:34:23
stack_traces                                                            NaN
stack_traces_remove                                                     NaN
pos_tagged_summary        {'stemmed': ['bug', 'illegalargumentexcept', '...
pos_tagged_description                     {'stemmed': [], 'unstemmed': []}
Name: 4, dtype: object

In [ ]:
report_df['pos_tagged_description'] = src_df['pos_tagged_description'].apply(ast.literal_eval)
report_df['pos_tagged_description'] = src_df['pos_tagged_description'].apply(lambda x: x['stemmed'])


In [34]:
def prepare_report_data():
    summary_contents = report_df[['key', 'pos_tagged_summary']].values.tolist()
    description_contents = report_df[['key', 'pos_tagged_description']].values.tolist()

    output_data = summary_contents
    for key, contents in description_contents:
        output_data[key].extend(contents)

    return output_data

In [35]:
report_data = prepare_report_data()

IndexError: list index out of range

In [67]:
def compute_lnc(query):
    tf_idf = defaultdict(float)

    term_freq = defaultdict(int)
    for term in query:
        term_freq[term] += 1

    for term, freq in term_freq.items():
        tf = 1 + math.log10(freq)
        tf_idf[term] = tf

    norm = math.sqrt(sum(weight ** 2 for weight in tf_idf.values()))
    if norm > 0:
        for term in tf_idf:
            tf_idf[term] /= norm

    return tf_idf


## RANKING RESULTS

In [95]:
def calculate_score(query, documents, inverted_index):
    query_vec = compute_lnc(query)
    doc_vecs = compute_ltc(documents, inverted_index)

    cosine_scores = {}

    for key, doc_vec in doc_vecs.items():
        dot_product = 0.0
        for term in query_vec:
            dot_product += query_vec[term] * doc_vec.get(term, 0.0) * coefficient_size[key]

        cosine_scores[key] = dot_product

    return cosine_scores

In [99]:
data = calculate_score(report_data[56012], data_src_codes_content, inverted_index_source_codes)

In [100]:
sorted_data = dict(sorted(data.items(), key=lambda item: item[1], reverse=True))

In [101]:
sorted_data

{'QueryTimeoutInterceptor': 0.035437979645814674,
 'StatementFinalizerTest': 0.03502756859000558,
 'Bug53545': 0.026336980939175626,
 'StatementFinalizer': 0.024291114946258435,
 'TesterFunctions': 0.02173546490731516,
 'TwoDataSources': 0.02120020678431368,
 'Direction': 0.02114291878182715,
 'TestQueryTimeoutInterceptor': 0.020030366360687345,
 'TestStatementCache': 0.01941816260507292,
 'StatementCounterInterceptor': 0.018758197058906318,
 'SocketNioValidateSend': 0.01754843343674862,
 'CreateTestTable': 0.01673001461463281,
 'AbstractCreateStatementInterceptor': 0.01607303635003223,
 'TestJspReader': 0.01601638832372792,
 'MultiPointSender': 0.01570376784482169,
 'SimplePOJOExample': 0.015617181239789599,
 'SocketNioSend': 0.015562260253930055,
 'Bug50571': 0.015521002733371647,
 'TagFileInfo': 0.01526249285200816,
 'TestException': 0.01509561523789896,
 'SimplePOJOAsyncExample': 0.014435143930611499,
 'Location': 0.014010673394535952,
 'TestJspDocumentParser': 0.013686000022862998